## 0. Instalacion de paquetes ###
La idea es setear el ambiente de codigo para poder empezar a trabajar con la consigna final de TP 

In [18]:
import pandas as pd # impotamos libreria pandas
import numpy as np #  importamos libreria numpy
import matplotlib as plt # importamos libreria matpoltlib
import seaborn as sns # importamos libreria seaborn
import scipy.stats as stats # importamos libreria scipy.stats
import pingouin as pg # importamos libreria pingouin

## 1. Carga del CSV con los datos electorales correspondientes ###
La idea es utilizar un csv de prueba para pdoer llevar adelante el analisis, el pedido del CSV se le hizo a ChatGPT para que pueda replicar el mismo

In [2]:
from pathlib import Path

# intentamos localizar el archivo en ../data con prefijo "encuesta"
data_dir = Path("../data")
if not data_dir.exists():
	raise FileNotFoundError(f"El directorio {data_dir.resolve()} no existe. Coloque el archivo dentro de ese directorio o ajuste la ruta.")

candidates = sorted(data_dir.glob("encuesta*"))
if not candidates:
	available = sorted([p.name for p in data_dir.iterdir()]) if data_dir.exists() else []
	raise FileNotFoundError(f"No se encontró ningún archivo con prefijo 'encuesta' en {data_dir.resolve()}. Archivos disponibles: {available}")

# preferir archivo .csv si existe, sino tomar el primero
csv_candidate = next((p for p in candidates if p.suffix.lower() == ".csv"), None)
file_to_read = csv_candidate or candidates[0]

# leemos el archivo csv y lo guardamos en un dataframe
df = pd.read_csv(file_to_read)
print(f"Leído: {file_to_read} (shape={df.shape})")

Leído: ..\data\encuesta100.csv (shape=(100, 10))


## 1.1 establecemos el rooting para que no suceda el mismo error de arriba de tener que no se encuentra el archivo y no se puede leer ###


## 2. Exploración y limpieza de datos ###
 Vemos el Dataframe y empezamos la limpieza como estandarizacion del mismo

In [3]:
df.shape  # mostramos la cantidad de filas y columnas del dataframe


(100, 10)

In [4]:
df.info()  # mostramos la informacion del dataframe

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 10 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   Fecha                                100 non-null    object
 1   Encuesta                             100 non-null    int64 
 2   Estrato                              100 non-null    object
 3   Sexo                                 100 non-null    object
 4   Edad                                 100 non-null    int64 
 5   Nivel Educativo                      100 non-null    object
 6   Cantidad de Integrantes en el Hogar  100 non-null    int64 
 7   Imagen del Candidato                 100 non-null    int64 
 8   Voto                                 100 non-null    object
 9   Voto Anterior                        100 non-null    object
dtypes: int64(4), object(6)
memory usage: 7.9+ KB


In [5]:
df.columns  # mostramos las columnas del dataframe

Index(['Fecha', 'Encuesta', 'Estrato', 'Sexo', 'Edad', 'Nivel Educativo',
       'Cantidad de Integrantes en el Hogar', 'Imagen del Candidato', 'Voto',
       'Voto Anterior'],
      dtype='object')

In [6]:
df.head()  # mostramos las primeras filas del dataframe

,Fecha,Encuesta,Estrato,Sexo,Edad,Nivel Educativo,Cantidad de Integrantes en el Hogar,Imagen del Candidato,Voto,Voto Anterior
0,2025-01-15,100,Alto,Femenino,33,Primario completo,2,87,Candidato A,No votó
1,2025-01-15,100,Bajo,Femenino,20,Primario incompleto,1,53,Candidato B,No votó
2,2025-01-15,100,Bajo,Masculino,63,Terciario/Universitario completo,6,74,En blanco,Candidato B
3,2025-03-01,400,Bajo,Femenino,69,Posgrado,1,88,Candidato B,En blanco
4,2025-02-15,300,Medio,Masculino,31,Posgrado,3,46,Candidato A,En blanco


In [7]:
df.tail()  # mostramos las ultimas filas del dataframe


,Fecha,Encuesta,Estrato,Sexo,Edad,Nivel Educativo,Cantidad de Integrantes en el Hogar,Imagen del Candidato,Voto,Voto Anterior
95,2025-02-01,200,Alto,Masculino,65,Secundario completo,3,86,No votó,No votó
96,2025-02-15,300,Alto,Masculino,36,Primario completo,3,51,Candidato C,Candidato A
97,2025-02-01,200,Medio,Masculino,80,Primario incompleto,5,58,Candidato B,En blanco
98,2025-01-15,100,Alto,Femenino,48,Secundario completo,4,61,Candidato B,Candidato A
99,2025-02-15,300,Medio,Masculino,70,Primario incompleto,4,71,Candidato A,No votó


In [8]:
df.sample(5)  # mostramos 5 filas aleatorias del dataframe

,Fecha,Encuesta,Estrato,Sexo,Edad,Nivel Educativo,Cantidad de Integrantes en el Hogar,Imagen del Candidato,Voto,Voto Anterior
73,2025-03-01,400,Medio,Masculino,78,Secundario incompleto,3,88,En blanco,En blanco
7,2025-02-01,200,Bajo,Masculino,20,Terciario/Universitario completo,2,89,Candidato C,Candidato A
69,2025-03-01,400,Bajo,Masculino,35,Secundario completo,1,85,Candidato C,Candidato B
52,2025-02-01,200,Medio,Femenino,37,Posgrado,2,54,Candidato A,Candidato B
82,2025-02-01,200,Medio,Masculino,55,Primario incompleto,3,76,En blanco,En blanco


In [9]:
df.dtypes  # mostramos los tipos de datos de cada columna

Fecha                                  object
Encuesta                                int64
Estrato                                object
Sexo                                   object
Edad                                    int64
Nivel Educativo                        object
Cantidad de Integrantes en el Hogar     int64
Imagen del Candidato                    int64
Voto                                   object
Voto Anterior                          object
dtype: object

In [10]:
df.describe()  # mostramos estadisticas descriptivas del dataframe

,Encuesta,Edad,Cantidad de Integrantes en el Hogar,Imagen del Candidato
count,100.000000,100.000000,100.000000,100.000000
mean,250.000000,48.580000,3.420000,73.250000
std,108.711461,18.207103,1.694793,17.151361
min,100.000000,18.000000,1.000000,40.000000
25%,200.000000,33.000000,2.000000,58.750000
50%,200.000000,47.500000,3.000000,76.500000
75%,300.000000,65.250000,5.000000,88.250000
max,400.000000,80.000000,6.000000,100.000000


In [11]:
df_2 = df.copy()  # hacemos una copia del dataframe original para trabajar sobre ella
df_2.columns = [col.lower().strip().replace(" ", "_") for col in df_2.columns]
for col in df_2.select_dtypes(include='object').columns:
    df_2[col] = (
        df_2[col]
        .str.strip()                               # eliminar espacios al inicio/final
        .str.lower()                               # pasar a minúsculas
        .str.replace(r'[^a-z0-9\s]', '', regex=True)  # quitar caracteres especiales
        .fillna('0')                               # reemplazar NaN por '0'
    )
for col in df_2.select_dtypes(include=['int64', 'float64']).columns:
    df_2[col] = df_2[col].fillna(0)

## 3. Análisis estadistico ####
La idea es poder llevar adelante los siguientes temas:
- Frecuencia y Distribución de datos
- MTC e Indices de Variabilidad
- Regresion
- Test de Hipotesis

## 3.1 Frecuencias y distibución de datos ### 

In [ ]:
# Tabla de frecuencias absolutas para varias columnas categóricas
print(df['columna_1'].value_counts())
print("----------------------------")
print(df['columna_2'].value_counts())
print("----------------------------")
print(df['columna_3'].value_counts())
print("----------------------------")
print(df['columna_4'].value_counts())

In [ ]:
# Tabla de frecuencia relativa para una columna categórica
print(df['columna_categorica'].value_counts(normalize=True))   # proporciones
df['columna_categorica'].value_counts(normalize=True) * 100     # porcentajes

In [ ]:
# Tabla de frecuencia absoluta con total incluido
df['columna_categorica'].value_counts().append(
    pd.Series({'Total': df['columna_categorica'].count()})

In [ ]:
# Tabla de contingencia de frecuencias absolutas
pd.crosstab(df['columna_1'], df['columna_2'])

In [ ]:
# Normalización respecto del total
print(pd.crosstab(df['columna_1'], df['columna_2'], normalize='all')
          .apply(lambda r: r / r.sum(), axis=1))
print("------------------------------------------------")

# Normalización por filas
print(pd.crosstab(df['columna_1'], df['columna_2'], normalize='index')
          .apply(lambda r: r / r.sum(), axis=1))
print("------------------------------------------------")

# Normalización por columnas
print(pd.crosstab(df['columna_1'], df['columna_2'], normalize='columns')
          .apply(lambda r: r / r.sum(), axis=0))

In [ ]:
# Tabla de contingencia de frecuencias absolutas con totales
print(pd.crosstab(df['columna_1'], df['columna_2'], margins=True))

print("----------------------------")

# Tabla de contingencia de frecuencias absolutas con totales incluyendo valores faltantes
print(pd.crosstab(df['columna_1'], df['columna_2'], margins=True, dropna=False))

In [ ]:
# Gráfico Q-Q para evaluar normalidad
stats.probplot(df['columna_numerica'], dist="norm", plot=plt)
plt.title("Normal Q-Q Plot")
plt.xlabel("Quantiles teóricos")
plt.ylabel("Quantiles muestrales")
plt.show()

########################
# Test de Shapiro-Wilk
########################
shapiro_test = shapiro(df['columna_numerica'])
print(f"Prueba de Shapiro-Wilk: W={shapiro_test.statistic}, p-value={shapiro_test.pvalue}")

########################
# Test de Kolmogorov-Smirnov
########################
ks_test = kstest(
    df['columna_numerica'],
    'norm',
    args=(np.mean(df['columna_numerica']), np.std(df['columna_numerica']))
)
print(f"Prueba de Kolmogorov-Smirnov: D={ks_test.statistic}, p-value={ks_test.pvalue}")

########################
# Plot de la distribución de frecuencia con histograma y curva de densidad
########################
plt.figure(figsize=(8,6))

# Histograma con curva de densidad
sns.histplot(data, kde=True, stat="density", color="blue", bins=30, label="Histograma")
sns.kdeplot(data, color="red", lw=2, label="Curva de densidad")

# Añadir la curva de una distribución normal teórica para comparar
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = stats.norm.pdf(x, np.mean(data), np.std(data))
plt.plot(x, p, 'k', lw=2, label="Distribución normal teórica")

# Personalización del gráfico
plt.title('Distribución de Frecuencia y Curva de Densidad')
plt.xlabel('Edad')
plt.ylabel('Densidad')
plt.legend()

plt.show()


In [ ]:
from matplotlib import gridspec

# Parámetros generales de ejemplo para distribuciones
np.random.seed(42)  # Para reproducibilidad

# Parámetros de distribuciones
param_poisson = 5
param_hipergeometrica = (10, 4, 5)   # (M, K, n)
param_t = 10                         # grados de libertad

# Generación de datos simulados
datos_poisson = np.random.poisson(param_poisson, 1000)
datos_hipergeom = np.random.hypergeometric(*param_hipergeometrica, 1000)
datos_t = np.random.standard_t(param_t, 1000)

# DataFrame para visualización
df = pd.DataFrame({
    'Poisson': datos_poisson,
    'Hipergeométrica': datos_hipergeom,
    'Distribución t': datos_t
})

# Función para construir Q-Q plots
def crear_qq_plot(serie, distribucion, parametros, eje):
    if distribucion == 'norm':
        stats.probplot(serie, dist='norm', plot=eje)
    elif distribucion == 't':
        stats.probplot(serie, dist='t', sparams=(parametros,), plot=eje)
    else:
        stats.probplot(serie, dist=distribucion, sparams=parametros, plot=eje)
    eje.set_title(f"Q-Q Plot: {distribucion}")

# Figura para Q-Q plots
fig = plt.figure(figsize=(15, 10))
grid = gridspec.GridSpec(2, 3, width_ratios=[1, 1, 1])

# Q-Q plots individuales
ax1 = plt.subplot(grid[0, 0])
crear_qq_plot(datos_poisson, 'poisson', (param_poisson,), ax1)

ax2 = plt.subplot(grid[0, 1])
crear_qq_plot(datos_hipergeom, 'hypergeom', param_hipergeometrica, ax2)

ax3 = plt.subplot(grid[0, 2])
crear_qq_plot(datos_t, 't', param_t, ax3)

plt.tight_layout()
plt.show()

# Distribuciones con histogramas y densidad
df_largo = pd.melt(df.reset_index(), id_vars=['index'], var_name='Distribución', value_name='Valor')

g = sns.FacetGrid(df_largo, col='Distribución', col_wrap=3, height=5)
g.map(sns.histplot, 'Valor', kde=True, bins=30, stat="density", alpha=0.6)
g.set_titles("{col_name}")
g.set_axis_labels("Valor", "Densidad")
g.add_legend()
plt.show()


## 3.2 MTC e Indices de Variabilidad ###

In [ ]:
# Cálculo de la moda 
valor_moda = df['columna_numerica'].mode()[0]
print(f"Moda: {valor_moda}")

In [ ]:
# Cálculo de la mediana 
valor_mediana = df['columna_numerica'].median()
print(f"Mediana: {valor_mediana}")

In [ ]:
# Calcular la Media Aritmética
valor_media = df['columna_numerica'].mean()
print(f"Media: {valor_media}")

In [ ]:
# Cálculo de cuartiles 
q1 = df['columna_numerica'].quantile(0.25)
q2 = df['columna_numerica'].quantile(0.50)  # Mediana
q3 = df['columna_numerica'].quantile(0.75)

# Mostrar resultados
print(f"Cuartiles: Q1={q1}, Q2={q2} (Mediana), Q3={q3}")

In [ ]:
# Cálculo de varianza y desviación estándar 
varianza = df['columna_numerica'].var()
desviacion_estandar = df['columna_numerica'].std()

print(f"Varianza: {varianza}")
print(f"Desviación estándar: {desviacion_estandar}")

In [ ]:
# Cálculo del coeficiente de variación para una columna numérica
coef_variacion = (desviacion_estandar / valor_media) * 100
print(f"Coeficiente de variación: {coef_variacion:.2f}%")

In [ ]:
# Cálculo del rango para una columna numérica
rango = df['columna_numerica'].max() - df['columna_numerica'].min()
print(f"Rango: {rango}")

In [ ]:
# Cálculo del rango intercuartílico para una columna numérica
rango_intercuartilico = stats.iqr(df['columna_numerica'])
print(f"Rango intercuartílico: {rango_intercuartilico}")

## 3.3 Regresión Lineal ###

In [ ]:
# Definición de variables independientes y dependientes
variable_x = 'columna_x'
variable_y = 'columna_y'

# Gráfico de dispersión
plt.scatter(df[variable_x], df[variable_y], label='Datos observados')

# Cálculo de la correlación de Pearson
coef_pearson = df[variable_x].corr(df[variable_y])
print(f'Correlación de Pearson entre {variable_x} y {variable_y}: {coef_pearson:.2f}')

# Ajuste del modelo de regresión lineal
X = sm.add_constant(df[variable_x])
modelo = sm.OLS(df[variable_y], X).fit()

# Generación de la recta de regresión
intercepto, pendiente = modelo.params
x_vals = np.linspace(df[variable_x].min(), df[variable_x].max(), 100)
y_vals = intercepto + pendiente * x_vals

# Inclusión de la recta de regresión en el gráfico
plt.plot(x_vals, y_vals, label=f'Recta de regresión (pendiente={pendiente:.2f})')

# Personalización del gráfico
plt.xlabel(variable_x)
plt.ylabel(variable_y)
plt.title('Gráfico de dispersión con recta de regresión')
plt.legend()
plt.show()


AttributeError: module 'matplotlib' has no attribute 'scatter'

## 3.5 Test de Hipotesis ###